In [4]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [5]:
#登入pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [6]:
#確認裡面有哪幾個database
clinet.database_names()

['admin',
 'iii-2017-07',
 'local',
 'mydb',
 'project',
 'rawData',
 'test',
 'test-database']

In [7]:
#選擇rawData這個db
db=clinet['rawData']

In [8]:
#確認裡面有哪幾個collection
db.collection_names()

['mobile01_tainan',
 'Tainan_final',
 'TainanUnique',
 'merge_tainanUni',
 'ptt_tainan_address',
 'AipinwanFood',
 'ptt_tainan_tel',
 'Tainan_final_name_id_1',
 'pixnet_tainan_author',
 'tbike',
 'ptt_tainan']

In [9]:
#將愛評網讀入
ipeenjson=list(db.Tainan_final.find({}))

In [7]:
ipeenjson[0]

{'_id': ObjectId('59afa8e6f3a322258443a5e3'),
 'address': '台南市安南區安北路433號',
 'average_consumption': '42',
 'business_hours': '09:00~23:00',
 'comment': [{'click': 1004,
   'content': '簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji

# 將結巴字典加入自己的許多詞

In [8]:

#結巴 匯入自己的詞典
jieba.set_dictionary('D:/WordLibrary/JiebaUse/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#店家種類字典introductionlist
introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
coordinatelist = [dien['coordinate'] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("D:\Data\csvtest\MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]


    

Building prefix dict from D:\WordLibrary\JiebaUse\jieba_dict.txt.big ...
Loading model from cache C:\Users\Java\AppData\Local\Temp\jieba.uf3aabe1eaf7c86dda8313189b0c5c683.cache
Loading model cost 1.035 seconds.
Prefix dict has been built succesfully.


In [9]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
diencommentlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [10]:
#各店家回覆dienmessagelist
dienmessagelist=[[''.join(j["message"]) for j in i["comment"] if j["message"]!=None and j["message"]!='null'] for i in ipeenjson]
dienmessagelist[0]

['感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950',
 '到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:)',
 '鍋燒意麵這款中山店沒有賣!!!!',
 '竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！',
 '好像我去過這家！很面熟，但是不太確定。',
 '回推囉，豆花看起來不錯呢！',
 '感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~',
 '豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!!',
 '每次去安平 我都必吃那家豆花耶~~~超愛!!',
 '我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享',
 '超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃',
 '好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※

In [11]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [12]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
dienallpinlist[0]


['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [13]:
# for megs in dienmessagelist:
#     for meg in megs:
#         if meg!=None and meg!='null'and meg!='':
#             print(" ".join(meg))

#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
messagecombinlist[0]

'感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950 到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:) 鍋燒意麵這款中山店沒有賣!!!! 竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！ 好像我去過這家！很面熟，但是不太確定。 回推囉，豆花看起來不錯呢！ 感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~ 豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!! 每次去安平 我都必吃那家豆花耶~~~超愛!! 我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享 超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃 好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※ 引述 品味廣東粥 之言：\t我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒

In [14]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
commentcombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [15]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
pincombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [16]:
#清一下記憶體
import gc
gc.collect()

0


# 將許多可能會用來分析的項目做成list

# 做TD-IDF分析

In [17]:

st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(pincombine, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and
                     '\u4e00' <= word and word <= '\u9fff' ])#and word not in dienlist and word not in pinlist
    pincutlist.append(pincut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for pincut in pincutlist:
        if word in pincut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

以切200篇文章的詞
以切400篇文章的詞
以切600篇文章的詞
以切800篇文章的詞
以切1000篇文章的詞
以切1200篇文章的詞
以切1400篇文章的詞
以切1600篇文章的詞
以切1800篇文章的詞
以切2000篇文章的詞
以切2200篇文章的詞
以切2400篇文章的詞
以切2600篇文章的詞
以切2800篇文章的詞
以切3000篇文章的詞
以切3200篇文章的詞
以切3400篇文章的詞
以切3600篇文章的詞
以切3800篇文章的詞
以切4000篇文章的詞
以切4200篇文章的詞
以切4400篇文章的詞
以切4600篇文章的詞
以切4800篇文章的詞
以切5000篇文章的詞
以切5200篇文章的詞
以切5400篇文章的詞
以切5600篇文章的詞
以計算1000個文字的出現篇數
以計算2000個文字的出現篇數
以計算3000個文字的出現篇數
以計算4000個文字的出現篇數
以計算5000個文字的出現篇數
以計算6000個文字的出現篇數
以計算7000個文字的出現篇數
以計算8000個文字的出現篇數
以計算9000個文字的出現篇數
以計算10000個文字的出現篇數
以計算11000個文字的出現篇數
以計算12000個文字的出現篇數
以計算13000個文字的出現篇數
以計算14000個文字的出現篇數
以計算15000個文字的出現篇數
以計算16000個文字的出現篇數
以計算17000個文字的出現篇數
以計算18000個文字的出現篇數
以計算19000個文字的出現篇數
以計算20000個文字的出現篇數
以計算21000個文字的出現篇數
以計算22000個文字的出現篇數
以計算23000個文字的出現篇數
以計算24000個文字的出現篇數
以計算25000個文字的出現篇數
以計算26000個文字的出現篇數
以計算27000個文字的出現篇數
以計算28000個文字的出現篇數
以計算29000個文字的出現篇數
以計算30000個文字的出現篇數
以計算31000個文字的出現篇數
以計算32000個文字的出現篇數
以計算33000個文字的出現篇數
以計算34000個文字的出現篇數
以計算35000個文字的出現篇數
以計算36000個文字的出現篇數
以計算37000個文字的出現篇數
以計算38000個文字的出現篇數
以計算39000個文字的出現篇數
以計算40000個文字的出現

# 將上面的斷詞文章combine在一起 做word2vec model

In [18]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(pincutlist)
with open('D:\Data\word2VecData\TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("D:/Data/word2VecData/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("D:/Data/word2VecData/Tainan.model.bin")


2017-09-19 12:54:38,595 : INFO : collecting all words and their counts
2017-09-19 12:54:38,595 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-09-19 12:54:41,786 : INFO : collected 221606 word types from a corpus of 12723068 raw words and 1273 sentences
2017-09-19 12:54:41,786 : INFO : Loading a fresh vocabulary
2017-09-19 12:54:42,168 : INFO : min_count=4 retains 83519 unique words (37% of original 221606, drops 138087)
2017-09-19 12:54:42,168 : INFO : min_count=4 leaves 12521166 word corpus (98% of original 12723068, drops 201902)
2017-09-19 12:54:42,356 : INFO : deleting the raw counts dictionary of 221606 items
2017-09-19 12:54:42,371 : INFO : sample=0.001 downsamples 20 most-common words
2017-09-19 12:54:42,371 : INFO : downsampling leaves estimated 12172301 word corpus (97.2% of prior 12521166)
2017-09-19 12:54:42,371 : INFO : estimated required memory for 83519 words and 300 dimensions: 242205100 bytes
2017-09-19 12:54:42,621 : INFO : resetting la

2017-09-19 12:55:56,249 : INFO : PROGRESS: at 29.10% examples, 243769 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:55:57,273 : INFO : PROGRESS: at 29.52% examples, 243894 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:55:58,289 : INFO : PROGRESS: at 29.91% examples, 243805 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:55:59,310 : INFO : PROGRESS: at 30.34% examples, 243886 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:56:00,358 : INFO : PROGRESS: at 30.73% examples, 243699 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:56:01,400 : INFO : PROGRESS: at 31.17% examples, 243880 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:56:02,432 : INFO : PROGRESS: at 31.59% examples, 243940 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:56:03,432 : INFO : PROGRESS: at 32.02% examples, 244119 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:56:04,452 : INFO : PROGRESS: at 32.43% examples, 244137 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:56:05,466 : INFO : PROGRESS: at 32.84% examples, 244162 wor

2017-09-19 12:57:18,743 : INFO : PROGRESS: at 61.85% examples, 242638 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:57:19,843 : INFO : PROGRESS: at 62.29% examples, 242654 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:57:20,856 : INFO : PROGRESS: at 62.70% examples, 242679 words/s, in_qsize 7, out_qsize 1
2017-09-19 12:57:21,921 : INFO : PROGRESS: at 63.11% examples, 242619 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:57:22,929 : INFO : PROGRESS: at 63.50% examples, 242585 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:57:23,940 : INFO : PROGRESS: at 63.91% examples, 242609 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:57:24,977 : INFO : PROGRESS: at 64.32% examples, 242589 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:57:25,989 : INFO : PROGRESS: at 64.70% examples, 242496 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:57:26,978 : INFO : PROGRESS: at 65.07% examples, 242414 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:57:28,015 : INFO : PROGRESS: at 65.50% examples, 242465 wor

2017-09-19 12:58:40,933 : INFO : PROGRESS: at 94.61% examples, 242640 words/s, in_qsize 7, out_qsize 1
2017-09-19 12:58:41,978 : INFO : PROGRESS: at 95.02% examples, 242611 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:42,985 : INFO : PROGRESS: at 95.43% examples, 242636 words/s, in_qsize 8, out_qsize 0
2017-09-19 12:58:44,003 : INFO : PROGRESS: at 95.85% examples, 242669 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:45,018 : INFO : PROGRESS: at 96.25% examples, 242648 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:46,042 : INFO : PROGRESS: at 96.64% examples, 242615 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:47,060 : INFO : PROGRESS: at 97.06% examples, 242662 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:48,081 : INFO : PROGRESS: at 97.45% examples, 242609 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:49,093 : INFO : PROGRESS: at 97.85% examples, 242594 words/s, in_qsize 7, out_qsize 0
2017-09-19 12:58:50,158 : INFO : PROGRESS: at 98.27% examples, 242593 wor

In [19]:
gc.collect()

0

# 試玩word2vec的model  需解除#

In [20]:
# from gensim.models import word2vec
# from gensim import models
# import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# model = models.Word2Vec.load('D:/Data/word2VecData/Tainan.model.bin')

# print("提供 3 種測試模式\n")
# print("輸入一個詞，則去尋找前一百個該詞的相似詞")
# print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
# print("輸入三個詞，進行類比推理")

# while True:
#     try:
#         query = input()
#         q_list = query.split()

#         if len(q_list) == 1:
#             print("相似詞前 30 排序")
#             res = model.most_similar(q_list[0],topn = 30)
#             for item in res:
#                 print(item[0]+","+str(item[1]))

#         elif len(q_list) == 2:
#             print("計算 Cosine 相似度")
#             res = model.similarity(q_list[0],q_list[1])
#             print(res)
#         else:
#             print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
#             res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
#             for item in res:
#                 print(item[0]+","+str(item[1]))
#         print("----------------------------")
#     except Exception as e:
#         print(repr(e))


# 試Rocchio、bayes、kmeans、SVM分群 (仿白話大數據寫法)

In [21]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(pincutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf

tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')



#ptt測試集|
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
# from sklearn.neighbors.nearest_centroid import NearestCentroid
st=time.time()
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)


#試用bayes分群
# from sklearn.naive_bayes import MultinomialNB
st=time.time()
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用kmeans分群
# from sklearn.neighbors import KNeighborsClassifier
st=time.time()
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Kmeans")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用svm分群
# from sklearn import svm
st=time.time()
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("svm")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

文字轉向量矩陣
tfidf
Roccio
array(['台菜餐廳', '美式料理', '義式料理', '西點烘焙、麵包', '其他鍋類', '', '臭豆腐', '', '複合式',
       '美式料理', '其他小吃', '泰式料理', '台菜餐廳'],
      dtype='<U13')
0.842841386795044
bayes
array(['其他小吃', '複合式', '複合式', '複合式', '複合式', '其他小吃', '其他小吃', '其他小吃', '複合式',
       '其他小吃', '其他小吃', '其他小吃', '其他小吃'],
      dtype='<U13')
1.6098763942718506
Kmeans
array(['複合式', '咖啡專賣', '其他小吃', '飲料專賣店', '其他小吃', '其他日式料理', '其他小吃', '其他小吃',
       '咖啡專賣', '咖啡專賣', '炭烤串燒', '其他小吃', '西點烘焙、麵包'],
      dtype='<U13')
0.46341896057128906
svm
array(['其他小吃', '複合式', '義式料理', '西點烘焙、麵包', '其他小吃', '其他小吃', '臭豆腐', '其他小吃',
       '複合式', '其他小吃', '其他小吃', '泰式料理', '其他小吃'],
      dtype='<U13')
253.99773383140564


# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > kmeans > bayes

In [22]:
#參考PTT斷詞文章
docs_new

['壹等賞 桃園 龍潭 看板 批 踢踢 實業 坊 批 踢踢 實業 坊 看板 聯絡 資訊 返回 看板 分享 作者 看板 標題 壹等賞 桃園 龍潭 時間 本文 轉錄 看板 作者 看板 標題 壹等賞 桃園 龍潭 時間 圖文 版 今天 提早到 新竹 火車站 附近 髮 瑟 剪 頭髮 整個 人 變得 清爽 免不了 念 幾句 想 以後 頭髮 剪 好 今天 比較 晚 下班 六點 多才 離開 公司 交通 正 處於 尖峰 時刻 光復路 繞 一圈 還了 漫畫 之後 決定 晚上 吃 桃園 龍潭 壹等賞 新竹 文化 街上 紅葉 告訴 目前 肚子 不餓 想 說 現在 開 壹等賞 時間 應該 夠長 肚子 應該 餓 決定 上次 曾經 吃過 一次 壹等賞 享用 晚餐 壹等賞 位於 龍潭 渴望 園區 附近 上次 來過 一次 不知 道路 走 知道 高速公路 累得 睡著 開 十分鐘 下 關西 交 流道 辦法 睡 一下 一下 交流 道 吵醒 請 幫 帶路 今 天 再 走過 一次 不太 認得 路 晚上 天色 黑 另一方 面是 山路 白天 去過 難記 住 路 走 下面 名片 背後 地圖 有興 趣 參考 一下 地址 桃園縣 龍潭 鄉 高原 村 中原 路 弄 電話 營業時間 每週 公休 今天 一到 停車場 滿滿的 車子 已經 七點 多了還 一堆 人 一看 發現 應該 公司 聚餐 很多人 穿著 同樣 制服 外面 抽 飯後 煙 空間 很大 晚到 位置 坐 壹等賞 一個 茶園 旁邊 一片 很大 茶 田 裡面 餐廳 賣 茶葉 東西 就連 洗手間 洗手 乳 茶葉 香味 服務 人員 一間 廂房 裡面 已經 坐著 滿滿的 客人 一到 看到 客人 本來 已經 心理準備 廚房 上菜 比較慢 一點 一點 完菜 不到 三分鐘 菜 一道 一道 送上來 真 佩服 廚房 裡面 大廚 燒菜 功力 上菜 食物 好吃 熱茶 免費 餐桌上 一壺 大壺 熱茶 免費 提供 飲用 醃菜 頭 免費 很喜歡 吃 只 吃 兩塊 剩下 吃 光光 用來 搭配 稀飯 一起 吃 最好 稀飯 點 好吃 茶油 拌 飯 珍香蜜 茶 鵝 小 茶鵝 很好吃 有點 貴 外面 賣 鵝肉 不太 應該 有用 茶葉 燻 很香 養生茶 油拌 飯 點了 兩碗 四碗 普通 碗 份量 要點 茶油 拌 飯 吃 哪有 吃白飯 道理 桂花 蜜 排骨 小 糖醋排骨 吃 覺得 還不錯 吃 最後 有點 吃 不

In [23]:
dienallpinlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [24]:
gc.collect()

0

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算5個)

In [25]:
#各店家評論diencommentlist
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            if len(re.findall(pinword,allpin))>0:
                if len(re.findall(pinword,allpin))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,allpin))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

已算完100篇情緒字
已算完200篇情緒字
已算完300篇情緒字
已算完400篇情緒字
已算完500篇情緒字
已算完600篇情緒字
已算完700篇情緒字
已算完800篇情緒字
已算完900篇情緒字
已算完1000篇情緒字
已算完1100篇情緒字
已算完1200篇情緒字
已算完1300篇情緒字
已算完1400篇情緒字
已算完1500篇情緒字
已算完1600篇情緒字
已算完1700篇情緒字
已算完1800篇情緒字
已算完1900篇情緒字
已算完2000篇情緒字
已算完2100篇情緒字
已算完2200篇情緒字
已算完2300篇情緒字
已算完2400篇情緒字
已算完2500篇情緒字
已算完2600篇情緒字
已算完2700篇情緒字
已算完2800篇情緒字
已算完2900篇情緒字
已算完3000篇情緒字
已算完3100篇情緒字
已算完3200篇情緒字
已算完3300篇情緒字
已算完3400篇情緒字
已算完3500篇情緒字
已算完3600篇情緒字
已算完3700篇情緒字
已算完3800篇情緒字
已算完3900篇情緒字
已算完4000篇情緒字
已算完4100篇情緒字
已算完4200篇情緒字
已算完4300篇情緒字
已算完4400篇情緒字
已算完4500篇情緒字
已算完4600篇情緒字
已算完4700篇情緒字
已算完4800篇情緒字
已算完4900篇情緒字
已算完5000篇情緒字
已算完5100篇情緒字
已算完5200篇情緒字
已算完5300篇情緒字
已算完5400篇情緒字
已算完5500篇情緒字
已算完5600篇情緒字
已算完5700篇情緒字
已算完5800篇情緒字
已算完5900篇情緒字
已算完6000篇情緒字
已算完6100篇情緒字
已算完6200篇情緒字
已算完6300篇情緒字
已算完6400篇情緒字
已算完6500篇情緒字
已算完6600篇情緒字
已算完6700篇情緒字
已算完6800篇情緒字
已算完6900篇情緒字
已算完7000篇情緒字
已算完7100篇情緒字
已算完7200篇情緒字
已算完7300篇情緒字
已算完7400篇情緒字
已算完7500篇情緒字
已算完7600篇情緒字
已算完7700篇情緒字
已算完7800篇情緒字
已算完7900篇情緒字
已算完8000篇情緒字
已算完8100篇情緒字
已算完8200篇情緒字
已算完8300篇情緒字
已算完8400篇情緒字
已

In [26]:
pinCountMaxper5list[50]

{'': 432,
 'Q彈': 1,
 '一品': 3,
 '一般般': 1,
 '不太喜歡': 2,
 '不好吃': 2,
 '不愛': 4,
 '不膩口': 4,
 '也不錯': 8,
 '乾了點': 1,
 '乾淨': 5,
 '亂': 5,
 '享受': 9,
 '便宜': 10,
 '偏酸': 1,
 '健康': 9,
 '入口即化': 1,
 '划算': 5,
 '剛好': 16,
 '口感紮實': 1,
 '古色古香': 1,
 '可愛': 20,
 '吃飽': 1,
 '合理': 2,
 '喜歡': 45,
 '大份': 2,
 '大碗': 3,
 '天然': 8,
 '天然食材': 2,
 '天龍': 4,
 '太多了': 3,
 '太甜': 14,
 '太臭': 1,
 '好吃': 96,
 '好喝': 4,
 '好好吃': 12,
 '好有味道': 2,
 '好甜': 3,
 '好貴': 3,
 '好食': 1,
 '實惠': 1,
 '寬敞': 1,
 '小巧': 4,
 '小貴': 1,
 '平價': 3,
 '平心而論': 1,
 '很不錯': 3,
 '很便宜': 3,
 '很喜歡': 7,
 '很好吃': 16,
 '很有禮貌': 1,
 '很棒': 1,
 '很清爽': 2,
 '很濃郁': 10,
 '很甜': 1,
 '很美味': 1,
 '很脆': 1,
 '很膩': 1,
 '很難吃': 1,
 '很香': 4,
 '後悔': 2,
 '態度好': 1,
 '整齊': 4,
 '新鮮': 11,
 '明亮': 5,
 '有禮貌': 1,
 '有落差': 1,
 '有點小': 2,
 '有點硬': 3,
 '柔軟': 3,
 '浪漫': 2,
 '清幽': 1,
 '清爽': 16,
 '溫暖': 3,
 '滿喜歡': 1,
 '滿意': 1,
 '漂亮': 4,
 '漲成': 1,
 '濃郁': 42,
 '熱呼呼的': 1,
 '熱情': 6,
 '爽口': 1,
 '特別': 21,
 '甜度恰到好處': 2,
 '精緻': 6,
 '精美': 3,
 '經濟實惠': 1,
 '綿密': 21,
 '綿密口感': 1,
 '綿綿的': 2,
 '缺陷': 1,
 '美味': 16,
 '脆脆的': 3,
 '膩口': 

In [27]:
#店家評論總數Ncommentlist
#店家回覆總數Nmessage
#將六篇回覆視為一篇文章，做正規化(防止評論數過多的店家分數極端化)
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [28]:
dienscorelist[2]

8.74269662921348

In [29]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [30]:
pinsynonymsdict

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'cp值高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美味',


In [31]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

# 將上述屬性結合成一個大分析表

In [32]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,coordinate in zip(dienlist,stylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,introductionlist,coordinatelist):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    
    dien['introduction']=introduction
    dien['coordinate']=coordinate
    
    
    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# pincutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper5list
# dienscorelist
# featurescores


#店家種類字典introductionlist
# introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
# coordinatelist = [dien['coordinate'] for dien in ipeenjson]

In [33]:
gc.collect()

0

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [34]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

重口味+0	5.0	0
重口味+5	1.9	0
重口味+10	1.3	0
重口味+15	1.3	0
偏甜+0	5.0	0
偏甜+5	1.3	0
偏甜+10	1.1	0
偏甜+15	1.7	0
食物精緻漂亮+0	15.0	0
食物精緻漂亮+5	7.2	0
食物精緻漂亮+10	4.5	0
食物精緻漂亮+15	6.0	0
偏酸+0	1.0	0
偏酸+5	0.5	0
偏酸+10	0.3	0
偏酸+15	0.4	0
食物美味+0	37.0	-19.0
食物美味+5	23.0	0
食物美味+10	19.9	0
食物美味+15	21.6	0
小份量+0	5.0	0
小份量+5	2.4	0
小份量+10	1.7	0
小份量+15	1.5	0
份量大+0	9.0	0
份量大+5	2.7	0
份量大+10	3.5	0
份量大+15	3.1	0
偏辣+0	5.0	0
偏辣+5	1.3	0
偏辣+10	0.9	0
偏辣+15	1.3	0
環境好+0	26.0	-3.0
環境好+5	6.7	-0.6
環境好+10	6.1	-0.5
環境好+15	5.3	-0.3
食物很香+0	8.0	0
食物很香+5	3.7	0
食物很香+10	2.3	0
食物很香+15	4.2	0
其他正面情緒+0	18.0	-2.0
其他正面情緒+5	7.5	-0.3
其他正面情緒+10	9.7	0
其他正面情緒+15	8.1	0
服務好+0	3.0	-3.0
服務好+5	1.9	-0.5
服務好+10	1.0	-0.2
服務好+15	0.9	-0.2
划算+0	14.0	-7.0
划算+5	5.5	-4.9
划算+10	5.1	-1.5
划算+15	2.5	-1.6
口味清淡+0	3.0	0
口味清淡+5	0.8	0
口味清淡+10	0.5	0
口味清淡+15	1.1	0


In [35]:
standard

{'份量大0': (4.5, 4.5),
 '份量大10': (1.75, 1.75),
 '份量大15': (1.55, 1.55),
 '份量大5': (1.35, 1.35),
 '偏甜0': (2.5, 2.5),
 '偏甜10': (0.55, 0.55),
 '偏甜15': (0.85, 0.85),
 '偏甜5': (0.65, 0.65),
 '偏辣0': (2.5, 2.5),
 '偏辣10': (0.45, 0.45),
 '偏辣15': (0.65, 0.65),
 '偏辣5': (0.65, 0.65),
 '偏酸0': (0.5, 0.5),
 '偏酸10': (0.15, 0.15),
 '偏酸15': (0.2, 0.2),
 '偏酸5': (0.25, 0.25),
 '其他正面情緒0': (8.0, 10.0),
 '其他正面情緒10': (4.85, 4.85),
 '其他正面情緒15': (4.05, 4.05),
 '其他正面情緒5': (3.6, 3.9),
 '划算0': (3.5, 10.5),
 '划算10': (1.7999999999999998, 3.3),
 '划算15': (0.44999999999999996, 2.05),
 '划算5': (0.2999999999999998, 5.2),
 '口味清淡0': (1.5, 1.5),
 '口味清淡10': (0.25, 0.25),
 '口味清淡15': (0.55, 0.55),
 '口味清淡5': (0.4, 0.4),
 '小份量0': (2.5, 2.5),
 '小份量10': (0.85, 0.85),
 '小份量15': (0.75, 0.75),
 '小份量5': (1.2, 1.2),
 '服務好0': (0.0, 3.0),
 '服務好10': (0.4, 0.6),
 '服務好15': (0.35, 0.55),
 '服務好5': (0.7, 1.2),
 '環境好0': (11.5, 14.5),
 '環境好10': (2.8, 3.3),
 '環境好15': (2.5, 2.8),
 '環境好5': (3.0500000000000003, 3.65),
 '重口味0': (2.5, 2.5),
 '重口味10': (0.65,

In [36]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [37]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
划算+0	1.3	-0.7
划算+5	1.1	-0.9
划算+10	1.5	-0.4
划算+15	1.2	-0.8
環境好+0	1.8	-0.2
環境好+5	1.8	-0.1
環境好+10	1.9	-0.1
環境好+15	1.9	-0.1
服務好+0	1.0	-1.0
服務好+5	1.6	-0.4
服務好+10	1.6	-0.4
服務好+15	1.6	-0.3
食物精緻漂亮+0	2.0	0
食物精緻漂亮+5	2.0	0
食物精緻漂亮+10	2.0	0
食物精緻漂亮+15	2.0	0
食物很香+0	2.0	0
食物很香+5	2.0	0
食物很香+10	2.0	0
食物很香+15	2.0	0
其他正面情緒+0	1.8	-0.2
其他正面情緒+5	1.9	-0.1
其他正面情緒+10	2.0	0
其他正面情緒+15	2.0	0
份量大+0	2.0	0
份量大+5	2.0	0
份量大+10	2.0	0
份量大+15	2.0	0
小份量+0	2.0	0
小份量+5	2.0	0
小份量+10	2.0	0
小份量+15	2.0	0
重口味+0	2.0	0
重口味+5	2.0	0
重口味+10	2.0	0
重口味+15	1.9	0
口味清淡+0	2.0	0
口味清淡+5	2.1	0
口味清淡+10	2.2	0
口味清淡+15	2.1	0
偏辣+0	2.0	0
偏辣+5	2.0	0
偏辣+10	1.9	0
偏辣+15	1.9	0
偏甜+0	2.0	0
偏甜+5	2.1	0
偏甜+10	2.0	0
偏甜+15	2.0	0
偏酸+0	2.0	0
偏酸+5	2.0	0
偏酸+10	2.1	0
偏酸+15	1.9	0


# 貼餐廳評價標籤!

In [38]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

同記安平豆花(安平總店)
['安平', '同記安平豆花', '紅豆', '服務較差', '珍珠', '豆花', '剉冰、豆花']
深藍咖啡館
['派', '栗子', '非常貴', '蛋糕', '深藍', '食物美味', '千層', '西點烘焙、麵包']
阿堂鹹粥
['價格較高昂', '虱目', '魚腸', '粥', '鹹粥', '阿堂', '食物美味', '其他小吃']
富盛號碗粿
['碗粿', '安平', '食物美味', '富盛號', '民族', '魚羹', '其他小吃']
依蕾特布丁奶酪(安平門市)
['典藏', '奶酪', '布丁', '依蕾特', '抹茶', '食物美味', '西點烘焙、麵包']
食下有約 想法廚房
['食物精緻漂亮', '想法廚房', '蹄膀', '區郡', '食下有約', '食物很香', '非常好吃', '環境好', '花雕', '其他正面情緒', '其他異國料理']
飛饗創意西式料理
['典藏', '食物精緻漂亮', '偏酸', '環境超棒', '燉', '饗', '土壤', '沙拉', '非常好吃', '服務好', '其他正面情緒', '義式料理']
莉莉水果店
['價格較高昂', '前路', '肉圓', '莉莉', '豆花', '水果', '食物美味', '剉冰、豆花']
連得堂餅舖
['價格較高昂', '連得堂', '餅家', '兩包', '崇安', '煎餅', '休閒零食']
矮仔成蝦仁飯
['價格較高昂', '食物美味', '蝦仁飯', '鴨蛋', '矮仔成', '蝦仁', '蛋湯', '其他小吃']
泰成水果店
['價格較高昂', '食物美味', '正興', '哈密瓜', '哈蜜瓜', '瓜瓜', '芒果', '果汁']
騷烤家(前鋒總店)
['騷烤', '燒烤', '府城騷烤家', '甜不辣', '油條', '非常好吃', '其他正面情緒', '炭烤串燒']
阿松割包
['價格較高昂', '割包', '豬舌', '豬舌包', '瘦肉', '花生醬', '食物美味', '麵食點心']
秉醇烘焙坊
['食物精緻漂亮', '瓜子', '餅乾', '火山', '奶酥', '食物很香', '其他正面情緒', '伯爵', '食物美味', '西點烘焙、麵包']
冰鄉
['食物美味', '草莓', '牛乳', '冰', '民生路', '芒

堆疊創意輕食、比利時啤酒
['堆疊', '食物精緻漂亮', '小份量', '水波', '燉雞', '赤崁樓', '包球', '食物美味', '美式料理']
阿湯哥手工米麻糬
['偏甜', '芝麻', '糬', '紫米', '阿湯哥', '麻', '其他正面情緒', '食物美味', '麵食點心']
土城海產店
['蟳', '土城', '蟹黃', '處女', '螃蟹', '食物美味', '海鮮餐廳']
曹師傅麵包坊
['丹麥', '服務好', '酒釀', '麵', '曹', '環境好', '食物美味', '桂圓', '西點烘焙、麵包']
石精臼蚵仔煎
['飯湯', '食物美味', '香菇', '蚵', '蚵仔煎', '民族', '其它中式料理']
木門朝午洋食 MOMENT
['木門', '朝午', '鬆餅', '洋食', '環境好', '帕尼尼', '食物美味', '其它主題餐廳']
來呷麵
['墊個', '魯味', '能用', '到後甲國', '補習', '麵攤、麵店']
TWIN coffee 咖啡 云
['早午餐', '價格較高昂', '偏酸', '庫克', '小份量', '鬆餅', '食物很香', '云', '非常好吃', '環境好', '咖啡', '複合式']
Dazzling Cafe(台南新光三越中山店)
['蜜糖', '價格較高昂', '卡士', '草莓', '義大利', '吐司', '食物美味', '複合式']
新都巷日本料理
['壽司', '大洋', '非常便宜', '竹定', '炸蝦', '份量大', '生魚片', '食物美味', '綜合日式料理']
方蘭川焦皮布丁
['安平', '窯', '焦皮', '焦糖', '布丁', '其他正面情緒', '食物美味', '西點烘焙、麵包']
Picturesque 塗鴉空間書店(長榮2店)
['食物美味', '小份量', '書店', '塗鴉', '環境好', '其他正面情緒', '野鴨', '空間', '帕里尼', '其它主題餐廳']
Daylight光合箱子
['早午餐', '光合', '價格較高昂', '豬小排', '箱子', '其他正面情緒', '吐司', '環境好', '食物美味', '複合式']
德斯啤鮮釀啤酒餐廳
['重口味', '啤酒', '豬腳', '食物精緻漂亮', '德斯', '食物很

麻古茶坊
['葡萄柚', '麻古', '服務好', '綠茶', '網路評論正面情緒非常多', '金萱', '非常好吃', '環境好', '雨神', '果汁']
瘋牛排洋食
['價格較高昂', '食物美味', '泰瑞莎', '份量大', '牛排', '育樂', '盎司', '瘋牛', '美式料理']
沾米廚房 Jami kitchen
['廚房', '神秘', '朕', '沾米', '環境好', '食物美味', '千層', '法式料理']
山林事務所
['小惠', '服務好', '古蹟', '南洋杉', '鬥雞眼', '小強', '咖啡專賣']
綠色空間(原怡平店)
['價格較高昂', '永華', '薇薇安', '食物美味', '綠色', '親子餐廳', '環境好', '空間', '咖啡專賣']
東咔滋日式定食屋(原東日本料理)
['定食', '食物美味', '環河', '咔', '其他正面情緒', '御定', '民族', '綜合日式料理']
田媽媽下營鵝肉美食店
['鵝', '食物美味', '大便臉', '服務較差', '鵝肉', '下營', '媽媽', '台菜餐廳']
丹丹漢堡(成功店)
['丹丹', '速食店', '麵線', '份量大', '丹丹漢堡', '划算', '南部', '食物美味', '漢堡、炸雞']
RÊVE Café 黑浮咖啡(台南永華店)
['價格較高昂', '永華', '份量大', '奶酒', '黑浮', '鬆餅', '食物很香', '其他正面情緒', '咖啡', '食物美味', '複合式']
浮游咖啡
['門外漢', '啤酒', '勁屋', '浮游', '環境比較不好', '咖啡館', '咖啡專賣']
芙烈達千層蛋糕
['卡士', '芙烈', '蛋糕', '深藍', '千層', '西點烘焙、麵包']
聖保羅烘焙花園(新營店)
['價格較高昂', '喜餅', '聖保羅', '重芋', '樂樂', '蛋糕', '西點烘焙、麵包']
疆疆串燒烤肉店
['烤肉', '羊肉串', '串燒', '疆疆', '卵小卷', '食物美味', '炭烤串燒']
杏本善
['食物精緻漂亮', '紅杏初', '訂票', '服務一級棒', '褔', '杏仁茶', '食物美味', '迪利', '飲料專賣店']
麗都大冰果
['價格較高昂', '十路', '麗都', '冰果

廚子貓
['妃', '焗', '夾餅', '滴', '舊誠品', '義式料理']
拿坡里披薩‧炸雞(台南東寧店)
['脆雞', '拿坡里', '小雞', '炸雞', '披薩', '披薩']
Caffebene海安藝文門市
['韓國', '拉麵', '刨冰', '咖啡', '韓式', '韓式料理']
不老莊藥膳香腸(本店)
['營業項目', '藥膳', '香腸', '老莊', '民族', '休閒零食']
南都石頭鄉玉米
['賭神', '南都', '鄉為', '石頭', '玉米', '炭烤串燒']
小北火雞肉飯
['無敵', '過貓', '吳郭魚', '冰梅', '企鵝', '其他小吃']
豐的海鮮漁府
['阿爸', '將軍', '環境比較不好', '粥', '丫爸', '螃蟹', '台菜餐廳']
甕王雞
['價格較高昂', '蒲小英', '二哥', '醉雞', '半隻', '桃子', '其它中式料理']
omaya(永康店)
['春川', '偏酸', '起司', '辛普森', '服務好', '韓式', '食物美味', '年糕', '韓式料理']
時蔬異理
['蒲小英', '石鍋', '素食', '葷食', '蛋包飯', '食物美味', '義式料理']
波哥茶飲
['蜜糖', '波哥', '脆笛', '吐司', '丹麥', '果汁']
盧家麵食里長乾麵
['餛飩', '屁用', '盧家', '划算', '麵', '里長', '食物美味', '麵攤、麵店']
鹿柴咖啡
['王維', '長頸鹿', '空山', '養鹿', '鹿柴', '咖啡專賣']
嘉義老牌迷你石頭火鍋
['韓城', '味精', '石頭火鍋', '鍋底', '火鍋', '涮涮鍋']
布萊恩紅茶(台南善化店)
['價格較高昂', '善化', '布萊恩', '阿薩姆', '鮮奶', '紅茶', '飲料專賣店']
大螃蟹海鮮碳烤
['月', '阿月', '螃蟹', '小四', '月媽', '台菜餐廳']
遠馨阿婆肉粽
['肉粽', '粽子', '端午節', '服務較差', '阿婆', '偷工減料', '其他小吃']
208巷碳烤
['價格較高昂', '燒烤', '老王', '秋刀魚', '食食', '噢', '熱炒、快炒']
阿派的店
['磚窯', '甕', '缸', '雞油', '啟蒙雞', '山產野菜餐廳']

['海產', '烹煮', '三股', '七股', '四季', '食物美味', '海鮮餐廳']
柴柴犬掌燒
['卡士', '阿云', '賣萌', '犬', '掌燒', '西點烘焙、麵包']
施家小卷米粉
['價格較高昂', '葉家', '小卷', '米粉', '施家', '高湯', '米粉湯']
屏息霜淇淋
['食物精緻漂亮', '屏息', '黑天鵝', '網路評論正面情緒非常多', '甜點', '環境好', '工坊', '霜淇淋', '食物美味', '冰淇淋']
裕益碗粿王
['碗粿', '演唱會', '五月天', '麻豆', '口里', '其它中式料理']
總理餐廳
['小燕子', '小飛俠', '不了情', '副總', '伯父', '台菜餐廳']
堤米和洋料理簡餐
['價格較高昂', '堤米', '主廚', '揪媽', '聯一', '茄汁', '義式料理']
郎港式鮮肉包(善化店)
['深坑', '善化', '郎港式鮮肉包', '食物不好吃', '深井', '鮮肉', '水煎包、生煎包']
台灣楊雞創意炸物專賣店
['楊雞', '鹹酥雞', '脆波', '份量大', '雞排', '麻鮮', '食物美味', '漢堡、炸雞']
正宗越南河粉
['牛肉', '湯跟河', '河粉', '春捲', '越南', '越南料理']
魷樂園
['熊熊', '正興', '魷魚', '哞', '高顏值', '炭烤串燒']
昭清宮平安素宴
['清宮', '昭', '善堂', '文衡', '佳里', '無評', '其它中式料理']
川蜀 四川麻辣燙
['肉盤', '買一送一', '黑川', '蜀', '鍋底', '食物很香', '其他正面情緒', '非常好吃', '划算', '麻辣火鍋']
白木屋(台南東寧店)
['千里眼', '小雞', '蛋糕', '老弟', '木屋', '西點烘焙、麵包']
台南古早味阿法紅豆
['泥球', '紅豆', '阿法', '貓子', '一球', '中式糕餅']
白苜養生館
['黑木耳', '誏', '美肌', '白木耳', '銀耳', '其他正面情緒', '甜品、甜湯']
Mon Ga Bagel
['酥餅', '摩門', '歐姆', '人造奶油', '海賊王', '複合式']
上品藥膳美食
['油飯', '咩', '雞酒', '藥膳', '五十', '其

['地震', '拉麵', '叉燒', '中華', '野崎家', '其他日式料理']
無季韓式館
['香菇', '小菜', '石鍋', '拌', '泡菜', '韓式料理']
BONJOUR帕里尼 輕食
['還沒烤', '中西區', '松露巧', '食物很香', '焦餅', '食物美味', '帕里尼', '其他速食']
御の饌
['寶寶', '相撲', '拉麵', '岐', '挑戰', '日式麵食專賣']
阿卿肉丸
['待查', '客客', '白河', '餓嚕', '征途', '麵食點心']
富霖海鮮餐廳--華平宴會館
['華平', '婚宴', '魚露', '館當', '富霖', '海鮮餐廳']
有機認證冬瓜蘆薈
['冬瓜', '品來', '黏液', '其他正面情緒', '蘆薈', '金桔', '飲料專賣店']
義式意適
['主廚', '蛤蠣', '焗', '文同', '濃湯', '義式料理']
山泉鹽水雞
['山泉', '友愛', '鹹水雞', '點至', '學姐', '鹹水雞']
13號素食
['推禿', '區興民街', '滷味', '茹素', '炒意', '中式']
煮的茶舖
['維宣', '炒飯', '小雞', '星期五', '素食', '其它中式料理']
58鍋燒
['料都給', '瘋鍋燒', '蛋汁秀', '店盛', '食物店', '其他小吃']
Roast Cafe(海安店)
['價格較高昂', '拖鞋', '羅斯特', '海安', '包內', '餐義式', '咖啡專賣']
薏仁館
['打成', '綠豆湯', '環境比較不好', '碎沙', '走回路', '那群', '剉冰、豆花']
味香鴨肉羹
['價格較高昂', '老記', '華味香', '羹', '新營', '鴨肉', '麵食點心']
森勝興蛋糕咖啡
['櫻桃', '此巧', '時照', '巧克力', '慕斯類', '咖啡專賣']
小倆口異國料理
['丟哪裡', '便當', '小雞', '左列', '主菜', '台菜餐廳']
愜愜私隅輕食茶飲
['愜私隅', '牛奶', '小雞', '愜', '再省', '咖啡專賣']
查理TEA SHOP
['上南英', '請水水', '查理', '人雅痞', '笨', '飲料專賣店']
瑪琪哈朵咖啡
['敦煌', '伯爵', '奶茶', '瑪琪', '哈朵', '

四海遊龍鍋貼專賣店
['價格較高昂', '底皮', '跟大骨', '玆', '牛肉麵', '鍋貼', '麵食點心']
侯門小院
['醫生', '老宅', '全成', '侯家', '醫師', '其他正面情緒', '熱炒、快炒']
來碗拉麵(麻豆店)
['回暖', '豚', '拉麵', '叉燒', '吃點熱', '划算', '日式麵食專賣']
仁壽宮傅家の蚵嗲
['咩', '歸仁', '食物美味', '豆乳', '仁壽', '五十', '其他小吃']
龍蟠鍋貼
['價格較高昂', '酸辣湯', '文敏', '林文敏', '城內', '鍋貼', '麵食點心']
阿川義豐冬瓜茶
['偏甜', '冬瓜', '哈密瓜', '鱔魚', '鱔魚意麵', '食物美味', '冬瓜茶', '飲料專賣店']
達美樂披薩 DOMINO'S PIZZA(歸仁店)
['達美樂', '酸甜', '餅皮', '食物很香', '鳳梨', '泰式', '披薩']
秀敏煎粿
['咩', '大成', '這篇內', '七元', '五十', '中式早餐']
漢棋薑母鴨
['薑母鴨', '咩', '歸仁', '漢棋', '五十', '薑母鴨']
市五鮮奶脆皮甜甜圈
['價格較高昂', '仔包', '燃料', '原物', '咖滋', '圈', '西點烘焙、麵包']
Mr.BREIZH 布雷茲先生
['寶寶', '號新', '食物不好吃', '需前', '可麗餅', '布雷', '西點烘焙、麵包']
無名紅燒牛肉麵
['紅燒', '林口', '北投區', '牛肉麵', '本原', '麵食點心']
阿西小吃店
['阿西', '除風濕', '鱔魚', '下得重', '私營', '熱炒、快炒']
丁丁紅茶冰
['價格較高昂', '冰火', '鐵路', '樂樂', '菠蘿油', '菠蘿包', '飲料專賣店']
千囍餅舖
['送禮', '路和府', '杏仁', '鳳梨', '片要', '中式糕餅']
CoCo都可(台南東寧)
['喝酸', '看診', '皮膚科', '波霸', '買多杯', '飲料專賣店']
小西腳鮮牛肉湯
['現宰外', '跟淋醬', '本店', '不油超', '介詔', '其他小吃']
阿彬牛肉湯
['三層', '上肉', '堂弟', '牛肉湯', '花肉', '其他小吃']
班鳩的窩(中華店)
['豬排'

['引擎', '汽修', '公車', '孩子', '保養', '其它中式料理']
伴你咖啡
['頭山孚佑宮', '奌', '馬公市', '羞於', '伴來', '複合式']
蒸鍋蒸煮健康鍋
['蒸', '蒸鍋', '環境比較不好', '芝心手', '這蒸', '龍膽', '其他鍋類']
好也
['場長', '蒸汽', '新化', '其他正面情緒', '順德', '桂田', '食物美味', '粵菜 港式飲茶']
david's 鮮果茶
['原茶', '覺醒', '檸茶', '抹茶', '鮮果', '果汁']
蜀姥香麻辣鍋(台南府前店)
['蜀姥香', '蜀', '前店', '麻辣鍋', '府', '食物美味', '麻辣火鍋']
好口味川粵館(中華西路店)
['無豬', '鍋巴', '小高', '夠鮮且', '食物很香', '大排檔', '粵菜 港式飲茶']
烤爽燒烤店
['正太', '惟獨', '燒烤', '袐', '卜蜂', '炭烤串燒']
台南桂田酒店-馥桂廳港式飲茶
['馥桂廳', '茶資', '韭菜', '酒店', '桂田', '粵菜 港式飲茶']
森品飲食空間
['餐廳', '拜拜', '家族', '小寶貝', '暖爐', '其它中式料理']
瑪莎園
['虎頭山', '玉井', '貓咪', '布丁', '風景', '咖啡專賣']
東門圓環頂陽春麵專家(東門圓環總店)
['點乾意', '三分之一', '圓環', '中舉', '抄手', '麵食點心']
炸的三角骨
['雞皮', '魷魚', '剪', '油機', '脫油過', '其他小吃']
二撰牛肉麵
['飽滴', '牛肉', '經我', '環境比較不好', '拉麵', '牛肉麵', '熱炒、快炒']
吳記金鍋貼
['可拿的給', '新國', '早點', '怎大間', '煎餃', '其他小吃']
何家鼎邊趖
['製成品', '銼料', '滑動', '趖', '鼎', '其他小吃']
黑美人肉燥飯
['葉子', '環境比較不好', '魚皮', '肉燥飯', '黑美人', '魚丸湯', '台菜餐廳']
武廟碗粿蔡
['碗粿', '蒜泥', '老闆娘', '武廟碗粿', '武廟', '其他小吃']
倍樂烘焙坊
['價格較高昂', '團購', '蒜味', '一包', '老哥', '新市', '西點烘焙、麵包']
黃家饅

['價格較高昂', '葉子', '虱目', '環境比較不好', '份量大', '阿堂', '阿憨', '油條', '服務好', '食物美味', '西式早餐']
烘廚
['', '無評', '熱炒、快炒']
早安美之城
['', '無評', '其他類型早餐']
橘子廚房
['鬆餅', '灣裡', '明興店', '類跟', '漢堡', '其他類型早餐']
名古屋涮涮鍋
['', '無評', '涮涮鍋']
吉星牛肉麵
['', '無評', '麵食點心']
老夫子牛肉麵(中山店)
['', '無評', '麵食點心']
朱員外拉麵店(台南佳里鎮)
['', '無評', '麵食點心']
回味茶站
['', '無評', '果汁']
早安！美芝城(台南活力東門)
['', '無評', '西式早餐']
旺獅手工詩目(台南佳里)
['', '無評', '其他亞洲料理']
麥當勞(台南永康大灣店)
['', '無評', '漢堡、炸雞']
九六臭豆腐
['', '無評', '其他小吃']
心情有約風尚咖啡
['', '無評', '咖啡專賣']
牛族牛排館
['', '無評', '美式料理']
牛家香麵館
['', '無評', '麵食點心']
台南夏林海鮮炭燒
['', '無評', '海鮮餐廳']
鼎盛火烤兩吃
['', '無評', '炭烤串燒']
蕃薯厝肉粿仔
['煎肉', '虱目', '西勢', '蕃薯', '粿', '其他小吃']
鄭家養生飲(小北夜市店)
['', '無評', '飲料專賣店']
西門粉圓冰店(東寧店)
['', '無評', '甜品、甜湯']
豆先生Mr.Bean
['', '無評', '剉冰、豆花']
A-Point Express 都會慢食 南紡購物中心
['', '無評', '墨西哥料理']
Crystal cafe 克里絲朵 茶飲專賣
['', '無評', '茶專賣']
綠色法藍絲
['火雞', '牛排', '鄉村', '攝影展', '洛林', '法式料理']
禿頭老爹鮑魚海產粥
['禿頭', '海產', '老爹', '粥', '鮑魚', '服務較差', '食物美味', '其他小吃']
Eating Time 義燉糖義式分享概念餐廳
['小毛', '麗婷', '義燉', '卡夫', '鍋較', '其它主題餐廳']
樹耳朵
['', '無評', '西點烘焙、麵包'

英倫威爾快樂親子城堡
['價格較高昂', '溜滑梯', '營運', '遊戲', '小桃', '飆速', '其它主題餐廳']
四海遊龍崇德店
['', '無評', '麵食點心']
韓佶 日韓式料理
['畫了顆', '絲海', '韓佶', '石鍋', '客鵬', '划算', '韓式料理']
貳木
['牙籤筒', '醬肉', '拉麵', '初剛', '點豚', '日式麵食專賣']
韓角村
['咩', '玫瑰', '特餐', '氣份', '快過', '韓式料理']
川仔關廟魯麵
['肉羹', '關廟', '魯', '大嘴巴', '噢', '麵攤、麵店']
老騎士街頭餐車
['焦樣', '倍', '墨西哥捲餅', '阿哲', '騎士', '墨西哥料理']
QQ甜不辣 豬腳飯
['', '無評', '其他小吃']
覓秘cafe
['價格較高昂', '藍晒圖', '宿舍', '覓秘咖啡', '國有財產', '文創', '複合式']
綠菓子
['', '無評', '飲料專賣店']
念念
['阿伯', '服務好', '水餃', '拉麵', '虎尾', '雞蛋糕', '其他正面情緒', '環境好', '食物美味', '西式早餐']
江蕙果汁
['纖維', '甜菜', '江蕙', '果汁', '服務好', '小恩', '果汁']
星月小火鍋
['軟殼', '主播', '食堂', '丼', '故事咖啡', '涮涮鍋']
明星牧場自營餐廳
['羊奶', '無骨', '牧場', '模型', '牛肉湯', '複合式']
原翠咖啡
['', '無評', '咖啡專賣']
Saminsu Cafe 公平貿易咖啡館
['', '無評', '咖啡專賣']
B仔港式鮮果冰品
['', '無評', '甜品、甜湯']
溫記手作麵
['', '無評', '其他美食']
政政麵館 政政小吃部
['', '無評', '眷村菜']
公不一漢堡店
['鐵門', '不一', '花椰菜', '漢堡', '編仔', '英式料理']
竹炭吉炭火燒肉(台南健康店)
['', '無評', '日式燒肉']
68手作奶茶
['', '無評', '飲料專賣店']
謝教授咖啡館
['', '無評', '複合式']
Enjoy火烤2鍋
['火烤兩吃', '烤肉', '軟絲', '桑妮', '划算', '復國', '食物美味', '火烤兩吃']

In [39]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]
# len(gooddien)

食下有約 想法廚房 34.61199999999995
{'食物美味': 1.36, '食物很香': 0.82, '環境好': 0.97, '食物精緻漂亮': 0.97, '份量大': 0.54, '其他正面情緒': 0.66, '划算': 0.18, '口味清淡': 0.07, '偏甜': 0.28, '服務好': 0.15, '重口味': 0.18, '小份量': 0.21, '偏辣': 0.06}


飛饗創意西式料理 36.16399999999993
{'食物美味': 1.33, '食物很香': 0.69, '划算': 0.31, '環境好': 1.06, '服務好': 0.51, '食物精緻漂亮': 1.28, '其他正面情緒': 0.85, '份量大': 0.65, '偏甜': 0.47, '口味清淡': 0.36, '偏酸': 0.8, '重口味': 0.31, '偏辣': 0.12, '小份量': 0.16}


小銅鍋義式餐廳(德安誠品店) 42.36956521739131
{'食物美味': 1.7, '食物很香': 0.99, '環境好': 1.13, '食物精緻漂亮': 0.71, '其他正面情緒': 0.62, '份量大': 0.65, '重口味': 0.45, '口味清淡': 0.24, '偏辣': 0.26, '划算': 0.18, '偏酸': 0.65, '服務好': 0.31, '偏甜': 0.11, '小份量': 0.17}


歐加里(台南大學店) 39.19999999999998
{'食物美味': 1.51, '食物很香': 0.72, '划算': 0.78, '環境好': 0.73, '食物精緻漂亮': 1.45, '其他正面情緒': 0.65, '份量大': 0.34, '小份量': 0.27, '重口味': 0.37, '偏辣': 0.12, '偏酸': 0.6, '口味清淡': 0.15, '服務好': 1.16, '偏甜': 0.33}


轉角餐廳 33.70909090909096
{'食物美味': 1.33, '食物很香': 1.0, '環境好': 1.01, '服務好': 0.6, '食物精緻漂亮': 0.82, '其他正面情緒': 0.65, '份量大': 0.33, '划算': -0.1, '重口味'

In [40]:
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json','w') as f:
    json.dump(BigAnalyzeTable,f)
with open('D:/Data/JsonData/TainanFood/TainanGoodDien_1.1.json','w') as f:
    json.dump(gooddien,f)   

In [41]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json') as f:
    data=json.load(f)

In [42]:
[dien['coordinate'] for dien in data]

[{'lat': 22.9999252, 'lng': 120.1534308},
 {'lat': 22.9841793, 'lng': 120.2185374},
 {'lat': 22.9903124, 'lng': 120.1981439},
 {'lat': 22.9973997, 'lng': 120.1991831},
 {'lat': 22.9980996, 'lng': 120.1741423},
 {'lat': 22.989468, 'lng': 120.162831},
 {'lat': 22.9910369, 'lng': 120.2214746},
 {'lat': 22.9890556, 'lng': 120.2041456},
 {'lat': 23.0004936, 'lng': 120.2068603},
 {'lat': 22.9891486, 'lng': 120.1951976},
 {'lat': 22.9945217, 'lng': 120.1971865},
 {'lat': 22.994548, 'lng': 120.2133713},
 {'lat': 22.9976135, 'lng': 120.1989103},
 {'lat': 23.0518138, 'lng': 120.1646187},
 {'lat': 22.9942969, 'lng': 120.2001762},
 {'lat': 22.973827, 'lng': 120.220684},
 {'lat': 22.9982344, 'lng': 120.174619},
 {'lat': 22.9933486, 'lng': 120.1976249},
 {'lat': 23.02912, 'lng': 120.2518763},
 {'lat': 22.9889141, 'lng': 120.1880278},
 {'lat': 22.9981748, 'lng': 120.1743296},
 {'lat': 22.9964763, 'lng': 120.2030135},
 {'lat': 22.9892031, 'lng': 120.1949238},
 {'lat': 22.9987503, 'lng': 120.1696348},
